# Libraries

In [1]:
import pandas as pd
import pyodbc
import os

# Parameters

In [2]:
path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\Colombia\Cruz Verde\2021'
anio = 2021
mes = 'Febrero'
filename_so = r'\Rotacion_CruzVerde_900199922_Consumo_Farma_A_202102.xlsx'
filename_st = r'\Inventario_CruzVerde_900199922_Consumo_Farma_A_202102.xlsx'

## Data Base

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

Nos traemos el primer domingo del mes Genomma:

In [7]:
qfecha = '''
SELECT MIN(TmpFecha) Fecha
FROM Gnm_DWH.dbo.Dim_Tiempo
WHERE TmpAnioSemanaGenomma = {0} AND TmpMesGenomma = {1} AND TmpNombreDiaSemana = 'Domingo';
            '''

# Load Data

**df_so**: Hace referecia a los datos de "Sell Out" <br>
**df_st**: Hace referecia a los datos de "Stock"

In [8]:
%%time
df_so = pd.read_excel(path + '\\' + mes + filename_so)
df_st = pd.read_excel(path + '\\' + mes + filename_st)

Wall time: 12.2 s


In [9]:
fecha = pd.read_sql(qfecha.format(anio, 2), conn1)

# Clean Data

## Sell Out

In [11]:
data_so = df_so[['Código Producto', 'Código de Barras', 'Descripción Producto', 'Codigo Org Inventario', 'Nombre Org Inventario', 'Unidades vendidas']].copy()

# Agregamos dos columnas nuevas
data_so['Cadena'] = 'Cruz Verde'
data_so['Fecha_Venta'] = fecha.loc[0, 'Fecha']

In [12]:
names_so = {'Código Producto':'Cod_Prod de la Cadena',
            'Código de Barras':'EAN o UPC Genomma',
            'Descripción Producto':'Descripcion_Prod',
            'Codigo Org Inventario':'Cod_Local',
            'Nombre Org Inventario':'Descripcion_Local',
            'Unidades vendidas':'Unidades'}

data_so.rename(names_so, axis=1, inplace=True)

In [13]:
data_so = data_so.pivot_table(index=['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena',
                                     'EAN o UPC Genomma', 'Descripcion_Prod',
                                     'Cod_Local', 'Descripcion_Local'],
                              values=['Unidades'], aggfunc='sum').reset_index()

In [15]:
data_so[:4]

,Fecha_Venta,Cadena,Cod_Prod de la Cadena,EAN o UPC Genomma,Descripcion_Prod,Cod_Local,Descripcion_Local,Unidades
0,2021-02-07,Cruz Verde,74125,650240004643,JABON EXFOL ANTI-IMPERFECC ASEPXIA BAR X 100GR,97,97_CRUZ_VERDE_CHICO,4.0
1,2021-02-07,Cruz Verde,74125,650240004643,JABON EXFOL ANTI-IMPERFECC ASEPXIA BAR X 100GR,98,98_CRUZ_VERDE_BELMIRA,5.0
2,2021-02-07,Cruz Verde,74125,650240004643,JABON EXFOL ANTI-IMPERFECC ASEPXIA BAR X 100GR,101,101_CRUZ _VERDE _DOMICILIOS _TOBERIN,1.0
3,2021-02-07,Cruz Verde,74125,650240004643,JABON EXFOL ANTI-IMPERFECC ASEPXIA BAR X 100GR,117,117_CRUZ_VERDE_POPAYAN,1.0


## Stocks

In [16]:
data_st = df_st[['ARTICULO', 'EAN', 'DESCRIPCION', 'CODIGO ORGANIZACION', 'ORGANIZACION', 'CANTIDAD ARTICULO']].copy()

# Agregamos dos columnas nuevas
data_st['Cadena'] = 'Cruz Verde'
data_st['Fecha_Stock'] = fecha.loc[0, 'Fecha']

In [17]:
names_st = {'ARTICULO':'Cod_Prod', 'EAN':'EAN o UPC Genomma',
            'DESCRIPCION':'Descripcion_Prod',
            'CODIGO ORGANIZACION':'Cod_Local',
            'ORGANIZACION':'Descripcion_Local',
            'CANTIDAD ARTICULO':'Unidades'}

data_st.rename(names_st, axis=1, inplace=True)

In [18]:
data_st = data_st.pivot_table(index=['Fecha_Stock', 'Cadena', 'Cod_Prod',
                                     'EAN o UPC Genomma', 'Descripcion_Prod',
                                     'Cod_Local', 'Descripcion_Local'],
                              values=['Unidades'], aggfunc='sum').reset_index()

In [19]:
data_st[:4]

,Fecha_Stock,Cadena,Cod_Prod,EAN o UPC Genomma,Descripcion_Prod,Cod_Local,Descripcion_Local,Unidades
0,2021-02-07,Cruz Verde,74122,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,129,129_CRUZ_VERDE_TUNJA,1.0
1,2021-02-07,Cruz Verde,74122,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,143,143_CRUZ_VERDE_FACATATIVA,1.0
2,2021-02-07,Cruz Verde,74122,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,454,454_CRUZ_VERDE_COLINA,2.0
3,2021-02-07,Cruz Verde,74122,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,564,564_CRUZ_VERDE_LA_MERCED,2.0


# Export to ISV

# Tranformation to report

In [24]:
data_st['EAN o UPC Genomma'] = data_st['EAN o UPC Genomma'].astype('int64')
data_st['EAN o UPC Genomma'] = data_st['EAN o UPC Genomma'].astype(str)

data_st['Cod_Local'] = data_st['Cod_Local'].astype('int64')
data_st['Cod_Local'] = data_st['Cod_Local'].astype(str)

data_so['EAN o UPC Genomma'] = data_so['EAN o UPC Genomma'].astype('int64')
data_so['EAN o UPC Genomma'] = data_so['EAN o UPC Genomma'].astype(str)
data_so['Cod_Local'] = data_so['Cod_Local'].astype('int64')
data_so['Cod_Local'] = data_so['Cod_Local'].astype(str)

data_st['ID'] = data_st['EAN o UPC Genomma'] + data_st['Cod_Local']
data_so['ID'] = data_so['EAN o UPC Genomma'] + data_so['Cod_Local']

data_sost = data_so.merge(data_st, on='ID', how='outer')

In [25]:
def fillna_numeric_columns(df, list_columns):
    data = df.copy()
    for column in list_columns:
        data[column] = data[column].fillna(value=0)
    return data

In [26]:
def fillna_nominal_columns(df, dict_equivalent):
    data = df.copy()
    for col1, col2 in dict_equivalent.items():
        data[col1] = data[col1].fillna(data[col2])
    return data

In [27]:
num_cols = ['Unidades_x', 'Unidades_y']
data_sost = fillna_numeric_columns(data_sost, num_cols)

In [28]:
nom_cols = {'Fecha_Venta':'Fecha_Stock',
            'Cadena_x':'Cadena_y',
            'Cod_Prod de la Cadena':'Cod_Prod', 
            'EAN o UPC Genomma_x':'EAN o UPC Genomma_y',
            'Descripcion_Prod_x':'Descripcion_Prod_y', 
            'Cod_Local_x':'Cod_Local_y',
            'Descripcion_Local_x':'Descripcion_Local_y'}

In [29]:
data_sost = fillna_nominal_columns(data_sost, nom_cols)

In [30]:
final = data_sost.pivot_table(index=['Fecha_Venta', 'Cadena_x', 'Cod_Prod de la Cadena', 
                                     'EAN o UPC Genomma_x', 'Descripcion_Prod_x',
                                     'Cod_Local_x','Descripcion_Local_x'],
                              values=['Unidades_x', 'Unidades_y'], aggfunc='sum').reset_index()

In [31]:
new_names = {'Fecha_Venta':'Fecha',
             'Cadena_x':'Cadena',
             'Cod_Prod de la Cadena':'Cod_Prod',
             'EAN o UPC Genomma_x':'EAN',
             'Descripcion_Prod_x':'Descripcion',
             'Cod_Local_x':'Cod_Local',
             'Descripcion_Local_x':'Local',
             'Unidades_x':'SO',
             'Unidades_y':'Inventario'}

final.rename(new_names, axis=1, inplace=True)

In [32]:
data_sost.shape

(14061, 17)

In [33]:
final.shape

(14061, 9)

In [34]:
final.isnull().sum()

Fecha          0
Cadena         0
Cod_Prod       0
EAN            0
Descripcion    0
Cod_Local      0
Local          0
SO             0
Inventario     0
dtype: int64

In [36]:
final[['SO', 'Inventario']].sum()

SO            22728.952381
Inventario    60816.190376
dtype: float64

In [37]:
final.columns

Index(['Fecha', 'Cadena', 'Cod_Prod', 'EAN', 'Descripcion', 'Cod_Local',
       'Local', 'SO', 'Inventario'],
      dtype='object')

In [ ]:
final.merge(precios, on='EAN o UPC Genomma', how='left')

In [48]:
final['EAN'] = final['EAN'].astype('int64')

In [49]:
final.merge(precios, on='EAN', how='left')

,Fecha,Cadena,Cod_Prod,EAN,Descripcion,Cod_Local,Local,SO,Inventario,LISTA
0,2021-02-07,Cruz Verde,74122.0,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,129,129_CRUZ_VERDE_TUNJA,0.0,1.0,NaN
1,2021-02-07,Cruz Verde,74122.0,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,143,143_CRUZ_VERDE_FACATATIVA,0.0,1.0,NaN
2,2021-02-07,Cruz Verde,74122.0,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,454,454_CRUZ_VERDE_COLINA,0.0,2.0,NaN
3,2021-02-07,Cruz Verde,74122.0,650240004681,ASEPXIA CAMOUFLAGE TUB X 28GR,564,564_CRUZ_VERDE_LA_MERCED,0.0,2.0,NaN
4,2021-02-07,Cruz Verde,74123.0,650240004001,ASEPXIA SPOT TUB X 28GR,171,171_CRUZ_VERDE_PLAZA_DE_LAS_AMERICAS,0.0,1.0,14375.0
...,...,...,...,...,...,...,...,...,...,...
14056,2021-02-07,Cruz Verde,167138.0,7798140259428,TOALLITAS EFECTO PURIFICANTE ASEPXIA PAQ X 2...,748,748_CRUZ_VERDE_PALMETTO,1.0,1.0,11529.0
14057,2021-02-07,Cruz Verde,167138.0,7798140259428,TOALLITAS EFECTO PURIFICANTE ASEPXIA PAQ X 2...,750,750_CRUZ_VERDE_CALLE 67,0.0,4.0,11529.0
14058,2021-02-07,Cruz Verde,167138.0,7798140259428,TOALLITAS EFECTO PURIFICANTE ASEPXIA PAQ X 2...,816,816_CRUZ_VERDE_SANTA_ROSA_DOMICILIOS,0.0,2.0,11529.0
14059,2021-02-07,Cruz Verde,167138.0,7798140259428,TOALLITAS EFECTO PURIFICANTE ASEPXIA PAQ X 2...,97,97_CRUZ_VERDE_CHICO,1.0,7.0,11529.0


In [ ]:
data_so